In [1]:
import torch
device = torch.device('cuda')

In [ ]:
import torchvision
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_trt2.pth'))

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)
camera.running = True
image = widgets.Image(format='jpeg', width=224, height=224)
park_slider = widgets.FloatSlider(description='park', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, park_slider]), speed_slider]))

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [ ]:
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg
import torch.nn.functional as F
import time

def update(change):
    global park_slider, robot
    new_image = change['new'] 
    
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[0])
    
    prob_park = float(output[0])
    
    park_slider.value = prob_park
    
    if prob_park < 0.3:
        car.throttle = 0
        time.sleep(1)
        car.throttle = -0.3
        time.sleep(1)
        car.throttle = -0
        time.sleep(1)
        car.steering = -1
        time.sleep(1)
        car.throttle = -0.3
        time.sleep(1.5)
        car.throttle = 0
        time.sleep(1)
        car.steering = 0
        time.sleep(1)
        car.throttle = -0.3
        time.sleep(1)
        car.throttle = 0
        time.sleep(1)
        car.steering = 1
        time.sleep(1)
        car.throttle = -0.3
        time.sleep(1)
        car.throttle = 0
        time.sleep(1)
        car.steering = -1
        time.sleep(1)
        car.throttle = 0.14
        time.sleep(1)
        car.throttle = 0
        time.sleep(1)
        car.steering = 0
        time.sleep(1)
        time.sleep(1)
        car.throttle = 0
        time.sleep(1)
        car.steering = 0
        time.sleep(1)
        car.throttle = 0.14
        time.sleep(1)
        car.throttle = -0.3
        time.sleep(1)
        car.throttle = -0
        time.sleep(1.2)
        car.throttle = -0
        car.steering = 0
        camera.unobserve(update, names='value')
    else:
        car.steering = 0.1
        car.throttle = speed_slider.value
        
update({'new': camera.value})  # we call the function once to initialize

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera